In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/oxml2023mlcases-esg-classifier/sample_submission.csv
/kaggle/input/oxml2023mlcases-esg-classifier/data/labels.csv
/kaggle/input/oxml2023mlcases-esg-classifier/data/reports/report_1834.pdf
/kaggle/input/oxml2023mlcases-esg-classifier/data/reports/report_1829.pdf
/kaggle/input/oxml2023mlcases-esg-classifier/data/reports/report_1253.pdf
/kaggle/input/oxml2023mlcases-esg-classifier/data/reports/report_1241.pdf
/kaggle/input/oxml2023mlcases-esg-classifier/data/reports/report_1145.pdf
/kaggle/input/oxml2023mlcases-esg-classifier/data/reports/report_810.pdf
/kaggle/input/oxml2023mlcases-esg-classifier/data/reports/report_567.pdf
/kaggle/input/oxml2023mlcases-esg-classifier/data/reports/report_833.pdf
/kaggle/input/oxml2023mlcases-esg-classifier/data/reports/report_311.pdf
/kaggle/input/oxml2023mlcases-esg-classifier/data/reports/report_433.pdf
/kaggle/input/oxml2023mlcases-esg-classifier/data/reports/report_1823.pdf
/kaggle/input/oxml2023mlcases-esg-classifier/data/reports/repor

In [2]:
!pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 71.2 MB/s eta 0:00:0000:0100:01


In [3]:
import numpy as np 
import pandas as pd 
!pip install -q PyMuPDF skimpy
import fitz
from pathlib import Path 
from PIL import Image
import skimpy
import plotly.express as px 
from glob import glob
import random
import matplotlib.pyplot as plt 
from wordcloud import WordCloud,STOPWORDS
from PIL import ImageColor
from tqdm import tqdm 
stopwords = set(STOPWORDS)

In [5]:
from pathlib import Path 

# directories & files
DIR_DATA = Path("/kaggle/input/oxml2023mlcases-esg-classifier/data")
REPORTS_DIR = "reports/"  #/kaggle/input/esgdata/data/reports
LABELS_FILE = "labels.csv"

# columns 
C_ID, C_CLASS = "id", "class"


In [7]:
import fitz
import pandas as pd 
from PIL import Image


def create_filepath(filename):
    return DIR_DATA / REPORTS_DIR / filename

def read_page(filename, page_number):
    filepath = create_filepath(filename)
    doc = fitz.open(filepath)
    page_index = page_number - 1
    page = doc.load_page(page_index) 
    return page.get_text()


def visualize_pdf_image(filename, page_number):
    doc = fitz.open(create_filepath(filename))
    page_index = page_number - 1
    pix = doc[page_index].get_pixmap()
    img_page = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    return img_page 

def get_labels():
    return pd.read_csv(DIR_DATA / LABELS_FILE)

def build_doc_id(filename, page_number): 
    return filename + "." + str(page_number)

In [8]:
# reports main report file
reports = pd.read_csv("/kaggle/input/oxml2023mlcases-esg-classifier/data/labels.csv")

reports.head(n=7)

,id,class
0,report_519.pdf.53,governance
1,report_1537.pdf.24,social
2,report_567.pdf.33,governance
3,report_1830.pdf.220,other
4,report_1253.pdf.46,governance
5,report_422.pdf.16,environmental
6,report_1824.pdf.94,other


In [9]:
import skimpy
skimpy.skim(reports)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 1956   │ │ string      │ 2     │                                                          │
│ │ Number of columns │ 2      │ └─────────────┴───────┘                                                          │
│ └───────────────────┴────────┘                                                                                  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name               ┃ NA      ┃ NA %       ┃ words per row                ┃ total words              ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩  │
│ │ id                        │       0 │          0 │                            1 │                     2000 │  │
│ │ class                     │       0 │          0 │                            1 │                     2000 │  │
│ └───────────────────────────┴─────────┴────────────┴──────────────────────────────┴──────────────────────────┘  │
╰────────────────────────────────────────────────────── End ──────────────────────────────────────────────────────╯

In [10]:
!pip install -U pypdfium2 wordcloud

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 39.9 MB/s eta 0:00:0000:0100:01


In [12]:
from wordcloud import WordCloud
from tqdm.auto import tqdm

import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd 
import pypdfium2 as pdfium

%matplotlib inline

In [13]:
def get_pdf(path):
    return pdfium.PdfDocument(path)

def get_content(page):
    textpage = page.get_textpage()
    return textpage.get_text_range()

def render(page):
    bitmap = page.render(
        scale = 1,    # 72dpi resolution
        rotation = 0, # no additional rotation
    )
    return bitmap.to_pil()

def extract_content_from_id(file_id: str) -> str :    
    # extract filename and page
    items = file_id.split('.')
    filename = '.'.join(items[:2])
    page_num = int(items[-1])-1
    
    # load pdf, select page, and extract its content
    filepath = os.path.join(report_path, filename)
    pdf = get_pdf(filepath)
    page = pdf[page_num]
    content = get_content(page)

    content = " ".join(content.lower().split())
    return content

In [15]:
basepath = "/kaggle/input/oxml2023mlcases-esg-classifier/data"
report_path = os.path.join(basepath, "reports")
label_path = os.path.join(basepath, "labels.csv")

In [16]:
df = pd.read_csv(label_path)

df.head(n=8)

,id,class
0,report_519.pdf.53,governance
1,report_1537.pdf.24,social
2,report_567.pdf.33,governance
3,report_1830.pdf.220,other
4,report_1253.pdf.46,governance
5,report_422.pdf.16,environmental
6,report_1824.pdf.94,other
7,report_1825.pdf.310,other


In [17]:
contents = []

for idx, row in tqdm(df.iterrows(), total=len(df)):
    file_id = row['id']
    content = extract_content_from_id(file_id)
    contents.append(content)
    
df['content'] = contents

  0%|          | 0/1956 [00:00<?, ?it/s]

In [18]:
df.head(n=6)

,id,class,content
0,report_519.pdf.53,governance,"2021 esg impact report data privacy, security,..."
1,report_1537.pdf.24,social,2021 sustainability report table of contents 2...
2,report_567.pdf.33,governance,sustainability governance clean harbors’ commi...
3,report_1830.pdf.220,other,220 report of the réviseur d’entreprises agréé...
4,report_1253.pdf.46,governance,46 pfizer 2021 esg report governance governanc...
5,report_422.pdf.16,environmental,callon petroleum | 2020 sustainability report ...


In [20]:
import spacy
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

# Load spaCy model
nlp = spacy.load('en_core_web_sm')

In [21]:
df=df[['class', 'content']]
df.head()

,class,content
0,governance,"2021 esg impact report data privacy, security,..."
1,social,2021 sustainability report table of contents 2...
2,governance,sustainability governance clean harbors’ commi...
3,other,220 report of the réviseur d’entreprises agréé...
4,governance,46 pfizer 2021 esg report governance governanc...


In [22]:
# Preprocess the text data
df['content'] = [' '.join([token.lemma_ for token in nlp(text)]) for text in df['content']]
# Transform the string labels to integer labels
label_encoder = LabelEncoder()
df['class'] = label_encoder.fit_transform(df['class'])

In [23]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df['content'], df['class'], test_size=0.25, random_state=42)

In [24]:
# Define a pipeline combining a text feature extractor with a simple classifier
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', RandomForestClassifier()),
])

# Fit the pipeline on the training set
pipeline.fit(X_train, y_train)

# Predict the labels on the test set
predictions = pipeline.predict(X_test)

# Evaluate the model
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.93      0.81      0.86        93
           1       0.93      0.72      0.81        60
           2       0.93      1.00      0.96       216
           3       0.84      0.91      0.88       120

    accuracy                           0.91       489
   macro avg       0.91      0.86      0.88       489
weighted avg       0.91      0.91      0.90       489



**Validate the results**

In [25]:
test_df = pd.read_csv('/kaggle/input/oxml2023mlcases-esg-classifier/sample_submission.csv')

In [27]:
test_df.head()

,id,class
0,report_1352.pdf.49,environmental
1,report_1835.pdf.143,environmental
2,report_1352.pdf.46,environmental
3,report_1179.pdf.47,environmental
4,report_607.pdf.12,environmental


In [29]:
!pip install PyPDF2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 8.5 MB/s eta 0:00:00


In [30]:
import PyPDF2
import os

In [35]:

import PyPDF2
import os

def read_pdf(file_path):
    with open(file_path, 'rb') as pdf_file_obj:
        pdf_reader = PyPDF2.PdfReader(pdf_file_obj)
        num_pages = len(pdf_reader.pages)
        text = ''
        for page in range(num_pages):
            page_obj = pdf_reader.pages[page]
            text += page_obj.extract_text()
    return text

# Prepare the correct path to the PDFs
pdf_directory = "/kaggle/input/oxml2023mlcases-esg-classifier/data/reports/"

# Remove the numbers after .pdf from the 'id' column
test_df['file_name'] = test_df['id'].str.split('.').str[0] + '.pdf'

# Add the directory path to the file name
test_df['file_path'] = pdf_directory + test_df['file_name']

# Read the text from each PDF and store it in a new 'content' column
test_df['content'] = test_df['file_path'].apply(read_pdf)


In [36]:
test_df.head()

,id,class,content,file_name,file_path
0,report_1352.pdf.49,environmental,2021 LG H&H \nESG REPORTReport Overview\nLG H...,report_1352.pdf,/kaggle/input/oxml2023mlcases-esg-classifier/d...
1,report_1835.pdf.143,environmental,ANNUAL REPORT 2018\n66560.indd 1 3/22/19 7...,report_1835.pdf,/kaggle/input/oxml2023mlcases-esg-classifier/d...
2,report_1352.pdf.46,environmental,2021 LG H&H \nESG REPORTReport Overview\nLG H...,report_1352.pdf,/kaggle/input/oxml2023mlcases-esg-classifier/d...
3,report_1179.pdf.47,environmental,FY 2021 ESG Disclosures\nBoldly moving forward...,report_1179.pdf,/kaggle/input/oxml2023mlcases-esg-classifier/d...
4,report_607.pdf.12,environmental,OUR PLANET PROMISE:\nA SUSTAINABLE FUTURE FOR ...,report_607.pdf,/kaggle/input/oxml2023mlcases-esg-classifier/d...


In [37]:
test_df=test_df[['content']]
test_df.head()

,content
0,2021 LG H&H \nESG REPORTReport Overview\nLG H...
1,ANNUAL REPORT 2018\n66560.indd 1 3/22/19 7...
2,2021 LG H&H \nESG REPORTReport Overview\nLG H...
3,FY 2021 ESG Disclosures\nBoldly moving forward...
4,OUR PLANET PROMISE:\nA SUSTAINABLE FUTURE FOR ...


In [ ]:
# Preprocess the text data
test_df['content'] = [' '.join([token.lemma_ for token in nlp(text)]) for text in test_df['content']]